In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import time
import datetime as dt
import os

In [2]:
constituents = pd.read_csv('New.csv')
constituents = constituents.drop_duplicates(['Symbol'])
try:
    constituents = constituents.drop(['Unnamed: 0'],axis=1)
except:
    pass
constituents

,Symbol,Name,Sector
0,A,Agilent Technologies,Health Care
1,AAL,American Airlines Gp,Industrials
2,AAP,Advance Auto Parts Inc,Consumer Discretionary
3,AAPL,Apple Inc,Information Technology
4,ABBV,Abbvie Inc,Health Care
...,...,...,...
500,XYL,Xylem Inc,Industrials
501,YUM,Yum! Brands,Consumer Discretionary
502,ZBH,Zimmer Biomet Holdings,Health Care
503,ZION,Zions Bancorporation,Financials


In [5]:
today = str(dt.date.today())[:5]+str(dt.date.today())[5:7]+str(dt.date.today())[7:10]
startday = str(dt.date.today()-dt.timedelta(1190))[:5]+str(dt.date.today()-dt.timedelta(1190))[5:7]+str(dt.date.today()-dt.timedelta(1190))[7:10]
yesterday = str(dt.date.today()-dt.timedelta(1))[:5]+str(dt.date.today()-dt.timedelta(1))[5:7]+str(dt.date.today()-dt.timedelta(1))[7:10]
stamp = input("Enter the date (YYYY-MM-DD): ")

start = startday
end = stamp

Enter the date (YYYY-MM-DD):  2019-12-12


In [6]:
today,startday,yesterday,stamp,start,end

('2019-12-13',
 '2016-09-09',
 '2019-12-12',
 '2019-12-12',
 '2016-09-09',
 '2019-12-12')

In [7]:
LIST = sorted(set(constituents['Symbol']))
print(Fore.YELLOW,'ON GENERE LaBase',Style.RESET_ALL)
tmps55=time.time()
LaBase = pd.DataFrame()
df = pd.DataFrame()
BADTICKER = ['AMCR']
for loop,ticker in enumerate(LIST):
#for loop in range(0, 2):
    try:
        df = web.DataReader(ticker,'yahoo',start,end)
        df['Date'] = df.index
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    except:
        BADTICKER.append(ticker)
        continue
    df['Symbol'] = ticker
    df = df[['Symbol','High','Low','Open','Close','Volume','Adj Close']]
    LaBase = pd.concat((LaBase,df),ignore_index=False)
tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"Time for completing LaBase Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

print('Il y a ',len(BADTICKER),' mauvais tickers')
print('Les voici')
print(BADTICKER)

#LaBase[LaBase.duplicated()].shape

print('Base générée : ',LaBase.shape)
print('Il y a ',LaBase[LaBase.duplicated()].shape,' Duplicats')
print('Hors AMCR, voici les duplicats') 
print(LaBase[(LaBase.duplicated()==True)&(LaBase['Symbol'] != 'AMCR')])

###########################################################
############## SI BESOIN DE RETIRER LA DATE DU JOUR #######
##
#  LaBase = LaBase[LaBase.index != '2019-09-18 00:00:00'] #
##
#
# LaBase.index = pd.to_datetime(LaBase['Date'],format='%Y-%m-%d')
###########################################################
try:
    os.remove('LaBase.csv')
except:
    pass
LaBase.to_csv('LaBase.csv') 

 ON GENERE LaBase 
 Time for completing LaBase Generation = 560.090000 seconds 
 
Il y a  4  mauvais tickers
Les voici
['AMCR', 'BF.B', 'BRK.B', 'TSS']
Base générée :  (409398, 7)
Il y a  (448, 7)  Duplicats
Hors AMCR, voici les duplicats
Empty DataFrame
Columns: [Symbol, High, Low, Open, Close, Volume, Adj Close]
Index: []


In [8]:
import talib
from sklearn.preprocessing import MinMaxScaler
import yaml
from keras.models import model_from_yaml
from sklearn.metrics import  precision_score
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
yaml.warnings({'YAMLLoadWarning': False})
warnings.filterwarnings("ignore")

#### DEF ####

def boost(df):
    X = df
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:-2,:-1]
    Xtest = X.iloc[-2:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:-2]
    ytest = yshift.iloc[-2:-1]

    model = xgb.XGBRegressor(n_estimators=20000, objective='reg:squarederror',learning_rate=1, gamma=1, subsample=1, colsample_bytree=1, max_depth=100)

    model.fit( Xtrain, ytrain, early_stopping_rounds=150, eval_set=[(Xtest, ytest)], verbose=0)

    ytrain_pred = model.predict(Xtrain)

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])

    df['Close.S'] = pred
    df['Close.S2'] = df['Close.S']
    df = df.dropna()
    return(df)

def prepa_data(df):
    rsi = talib.RSI(df['Close'],timeperiod=14)
    stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
    upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
    sma5 = talib.SMA(df['Close'],timeperiod=5)
    sma8 = talib.SMA(df['Close'],timeperiod=8)
    sma10 = talib.SMA(df['Close'],timeperiod=10)
    sma12 = talib.SMA(df['Close'],timeperiod=12)
    sma15 = talib.SMA(df['Close'],timeperiod=15)
    sma30 = talib.SMA(df['Close'],timeperiod=30)
    sma35 = talib.SMA(df['Close'],timeperiod=35)
    sma40 = talib.SMA(df['Close'],timeperiod=40)
    sma45 = talib.SMA(df['Close'],timeperiod=45)
    sma50 = talib.SMA(df['Close'],timeperiod=50)
    atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
    delta5_8 = sma5 - sma8
    delta8_10 = sma8 - sma10
    delta10_12 = sma10 - sma12
    delta12_15 = sma12 - sma15
    delta15_30 = sma15 - sma30
    delta30_35 = sma30 - sma35
    delta35_40 = sma35 - sma40
    delta40_45 = sma40 - sma45
    delta45_50 = sma45 - sma50
    bbdelta = upper - middle
    price_bolup = df['Close'] - lower
    price_bolow = df['Close'] - upper
    Ema = talib.EMA(df['Close'],timeperiod=20)
    KC_High = Ema + 2*atr
    KC_Low = Ema - 2*atr
    aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
    aroon = aroonup - aroondown 
    rsi30_list = []
    rsi70_list = []
    for i in range(0,df.shape[0]):
        
        rsi70_list.append(70 - rsi.iloc[i])
        rsi30_list.append(rsi.iloc[i] - 30)
        
    varop_spy = df['Open'] - df['Close']
    varhl_spy = df['High'] - df['Low']
    df['Varop_Spy'] = varop_spy
    df['Varhl_spy'] = varhl_spy
    df['RSI'] = rsi
    df['70 - RSI'] = np.array(rsi70_list)
    df['RSI - 30'] = np.array(rsi30_list)
    df['BBD_Delta_Up'] = bbdelta
    df['delta5_8'] = delta5_8
    df['delta8_10'] = delta8_10
    df['delta10_12'] = delta10_12
    df['delta12_15'] = delta12_15
    df['delta15_30'] = delta15_30
    df['delta30_35'] = delta30_35
    df['delta35_40'] = delta35_40
    df['delta40_45'] = delta40_45
    df['delta45_50'] = delta45_50
    df['Stoc_Slowk'] = stoc_slowk
    df['Stoc_Slowd'] = stoc_slowd
    df['KC_High'] = KC_High
    df['KC_Low'] = KC_Low
    df['upper'] = upper
    df['lower'] = lower
    df['var_bollup_kchigh'] = upper-KC_High
    df['var_bolllow_kclow'] = lower-KC_Low
    df['Aroon Up'] = aroonup
    df['Aroon Down'] = aroondown
    df['Delta Aroon'] = aroon
    up = []
    down = []
    df = df.dropna()
    #df = boost(df)
    df['%Futur'] = ((df['Close.S']-df['Close']) *100) / (df['Close'])
    df['%Futur2'] = ((df['Close.S2']-df['Close']) *100) / (df['Close'])
    for i in range(0,df.shape[0]):
        if df.iloc[i]['%Futur'] > 0.5 :
            up.append(1)
            down.append(0)
        elif df.iloc[i]['%Futur'] < -0.5:
            up.append(0)
            down.append(1)
        else:
            up.append(0)
            down.append(0)
        
    df['target_up'] = up  
    df['target_down'] = down 
    #df = df.dropna()
    
    df = df[['High','Low','Open','Volume','Close','Varop_Spy','Varhl_spy','RSI',\
             '70 - RSI','RSI - 30','BBD_Delta_Up','delta5_8','delta8_10','delta10_12',\
             'delta12_15','delta15_30','delta30_35','delta35_40','delta40_45','delta45_50',\
             'Stoc_Slowk','Stoc_Slowd','KC_High','KC_Low','upper','lower','var_bollup_kchigh',\
             'var_bolllow_kclow','Aroon Up','Aroon Down','Delta Aroon','Close.S','Close.S2',\
             '%Futur','%Futur2','target_up','target_down']]
    
    return(df)

def deep_learning(df):
    X = df.copy()
    #####################X = X.iloc[:,1:-4]
    X = X.iloc[:,:-4]
    X.astype(np.float64)
    y_up = df.iloc[:,-2].values
    y_down = df.iloc[:,-1].values

    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_up = np.array(y_up).reshape(-1,1)
    y_down = np.array(y_down).reshape(-1,1)

    Xtrain = X[:bloc1,:]
    Xtest = X[bloc1:,:]
    ytrain_up = y_up[:bloc1,:]
    ytest_up = y_up[bloc1:,:]
    ytrain_down = y_down[:bloc1,:]
    ytest_down = y_down[bloc1:,:]

    seed = 770
    np.random.seed(seed)

    ytrain_up = ytrain_up.reshape(ytrain_up.shape[0],)
    ytrain_down = ytrain_down.reshape(ytrain_down.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

    yhat_up = model_up.predict_classes(Xtest)
    yhat_down = model_down.predict_classes(Xtest)

    predict_up = model_up.predict(Xtest)
    predict_down = model_down.predict(Xtest)

    precision_up = precision_score(ytest_up, yhat_up)  
    precision_down = precision_score(ytest_down, yhat_down) 

    resultats = pd.DataFrame()
    resultats['Date'] = df.index[bloc1:]
    resultats.index= df.index[bloc1:]
    resultats['Move Up'] = yhat_up
    resultats['Confiance up'] = (predict_up)*100
    resultats['Move Down'] = yhat_down
    resultats['Confiance Down'] = (predict_down)*100
    resultats['Actual'] = df.iloc[bloc1:]['Close']
    resultats['Actual.S'] = df.iloc[bloc1:]['Close.S']
    open_S = df['Open'].shift(-1)
    resultats['Open.S'] = open_S.iloc[bloc1:]
    dmp_cp=[]
    dmp_cp = ((resultats['Confiance up']-resultats['Confiance Down'])/(resultats['Confiance up']+resultats['Confiance Down'])*100)
    resultats['DMP_CP'] = dmp_cp
    
    return(resultats,precision_up,precision_down,scaler)

GREEN = []
RED = []
GREEN_CLOSE = []
RED_CLOSE=[]
TICKER = []
DATE = []
PRICE = []
i = -1

tmps55=time.time()
print('The computing has begun at ',dt.datetime.now().hour,':',dt.datetime.now().minute,':',dt.datetime.now().second)
print('Le denier signal est ',Fore.YELLOW,LaBase.index[-1],Style.RESET_ALL)
LeChemin = 'DONNEES_DL/'

list_dir = os.listdir(LeChemin+'.')
NEW_LIST = []
for pikunichihouahoua in range(0,len(str(list_dir).split(','))):
    if str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[1].split("'")[0]=='yaml':
        NEW_LIST.append(str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[0].split("'")[1].split('_')[1])
NEW_LIST = sorted(set(NEW_LIST))

for loop,ticker in enumerate(NEW_LIST):
    
    if loop == 1:
            print('\r',Fore.GREEN,' |==',Fore.BLUE,'O',Fore.RED,'==================|',end='',flush=True)
    if loop == 50:
        print('\r',Fore.GREEN,' |====',Fore.BLUE,'O',Fore.RED,'================|',end='',flush=True)
    if loop == 100:
        print('\r',Fore.GREEN,' |======',Fore.BLUE,'O',Fore.RED,'==============|',end='',flush=True)
    if loop == 150:
        print('\r',Fore.GREEN,' |========',Fore.BLUE,'O',Fore.RED,'============|',end='',flush=True)
    if loop == 200:
        print('\r',Fore.GREEN,' |==========',Fore.BLUE,'O',Fore.RED,'==========|',end='',flush=True)
    if loop == 250:
        print('\r',Fore.GREEN,' |============',Fore.BLUE,'O',Fore.RED,'========|',end='',flush=True)
    if loop == 300:
        print('\r',Fore.GREEN,' |==============',Fore.BLUE,'O',Fore.RED,'======|',end='',flush=True)
    if loop == 350:
        print('\r',Fore.GREEN,' |================',Fore.BLUE,'O',Fore.RED,'====|',end='',flush=True)
    if loop == 400:
        print('\r',Fore.GREEN,' |==================',Fore.BLUE,'O',Fore.RED,'==|',Style.RESET_ALL,end='',flush=True)
    

    yamlup = ('DONNEES_DL/Save_'+ticker+'_up.yaml')
    yamldown = ('DONNEES_DL/Save_'+ticker+'_down.yaml')
    modelup = ('DONNEES_DL/Save_'+ticker+'_up.h5')
    modeldown = ('DONNEES_DL/Save_'+ticker+'_down.h5')

    yaml_file_up = open(yamlup, 'r')
    yaml_file_down = open(yamldown, 'r')
    model_yaml_up = yaml_file_up.read()
    model_yaml_down = yaml_file_down.read()
    yaml_file_up.close()
    yaml_file_down.close()
    model_up = model_from_yaml(model_yaml_up)
    model_down = model_from_yaml(model_yaml_down)
    # load weights into new model
    model_up.load_weights(modelup)
    model_down.load_weights(modeldown)


    #df = scrap_data(ticker,start,end)
    
    df = LaBase[LaBase['Symbol']==ticker]
    df = df.drop(['Symbol'],axis=1)
    #df = df.iloc[-820:,:]
    df = df.drop(['Close'],axis=1)
    df['Close'] = df['Adj Close']
    df = df.drop(['Adj Close'],axis = 1)
    df = boost(df)

    delta = round(df.shape[0])
    bloc1 = round(delta*0.80)
    bloc2 = delta - bloc1

    df = prepa_data(df)

    resultats,precision_up,precision_down,scaler = deep_learning(df)
    
    price = df.iloc[i]['Close']

    if (precision_up * 100) > 69 and (precision_down * 100) > 69:

        ####################
        ##### SIGNALS #####
        ###################

        filtre_up = 93
        filtre_down = 97
        filtre_up_close = 80
        filtre_down_close = 80


        if resultats.iloc[i]['Confiance Down'] > filtre_down:
            doob = -1
        elif resultats.iloc[i]['Confiance Down'] > filtre_down_close and resultats.iloc[i]['Confiance Down'] < filtre_down :
            waab = -1
            doob = 0
        elif resultats.iloc[i]['Confiance up'] > filtre_up:
            doob = 1
        elif resultats.iloc[i]['Confiance up'] > filtre_up_close and resultats.iloc[i]['Confiance up'] < filtre_up:
            waab = 1
            doob = 0
        else :
            doob = 0
            waab = 0

        if doob == 1 :
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(1)
            RED.append(0)
            GREEN_CLOSE.append(0)
            RED_CLOSE.append(1)
            PRICE.append(price)

        elif doob == -1 :
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(0)
            RED.append(1)
            GREEN_CLOSE.append(1)
            RED_CLOSE.append(0)
            PRICE.append(price)

        elif doob == 0 and waab == -1 :
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(0)
            RED.append(0)
            GREEN_CLOSE.append(1)
            RED_CLOSE.append(0)
            PRICE.append(price)

        elif doob == 0 and waab == 1 :
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(0)
            RED.append(0)
            GREEN_CLOSE.append(0)
            RED_CLOSE.append(1)
            PRICE.append(price)

        else:
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(0)
            RED.append(0)
            GREEN_CLOSE.append(0)
            RED_CLOSE.append(0)
            PRICE.append(price)

    else:
        DATE.append(df.index[i])
        TICKER.append(ticker)
        GREEN.append(0)
        RED.append(0)
        GREEN_CLOSE.append(0)
        RED_CLOSE.append(0)
        PRICE.append(price)
signals = pd.DataFrame(columns = ['Date','Ticker','Open Long','Close Long','Open Short','Close Short'])            
signals['Date'] = DATE
signals['Ticker'] = TICKER
signals['Open Long'] = GREEN
signals['Close Long'] = GREEN_CLOSE
signals['Open Short'] = RED
signals['Close Short'] = RED_CLOSE 
signals['Close Price'] = PRICE

try:    
    signals = pd.read_csv('SIGNALS_'+stamp+'.csv')
    try:
        signals = signals_csv.drop('Unnamed: 0',axis=1)
    except:
        pass
    print('Le fichier SIGNALS_'+stamp+'.csv existe déjà')
        
except:
    print('\n Création du fichier SIGNALS_'+stamp+'.csv')
    try:
        signals = signals_csv.drop('Unnamed: 0',axis=1)
    except:
        pass
    signals = signals[(signals['Close Long']==1)|(signals['Close Short']==1)].drop_duplicates()
    signals.to_csv('SIGNALS/SIGNALS_'+stamp+'.csv')        

tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"Time for complete Signals Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

Using TensorFlow backend.


The computing has begun at  11 : 1 : 13
Le denier signal est   2019-12-12 00:00:00 
   |==============  O  ======|
 Création du fichier SIGNALS_2019-12-12.csv
 Time for complete Signals Generation = 382.940000 seconds 
 


In [9]:
len(signals)

45

In [10]:
signals[(signals['Close Long']==1)|(signals['Close Short']==1)].drop_duplicates()

,Date,Ticker,Open Long,Close Long,Open Short,Close Short,Close Price
5,2019-12-12,ACN,0,1,0,0,203.919998
17,2019-12-12,AMCR,0,1,1,0,10.680000
34,2019-12-12,BAC,0,1,1,0,34.680000
50,2019-12-12,CAH,0,1,1,0,54.660000
69,2019-12-12,COF,0,1,1,0,104.029999
72,2019-12-12,COP,0,1,0,0,62.590000
88,2019-12-12,DISCA,0,1,1,0,32.360001
90,2019-12-12,DLR,0,1,1,0,114.669998
91,2019-12-12,DOV,0,1,1,0,114.480003
96,2019-12-12,DVN,0,1,1,0,23.990000


In [11]:
import ib_insync
from ib_insync import *
import sys

In [12]:
ib = IB()
log = ''
compte = 'DU1507426'
os.system('say "Le golem pour te servir, mon maitre. Hoilleli, hoille vavhoille li, mon bon maitre, si je ne te sers pas ou si je ne remplis pas ma mission"')

0

In [14]:
def conX():
    util.startLoop()  # uncomment this line when in a notebook
    ib.connect('127.0.0.1', 7497, clientId=1)
    if ib.isConnected() == True:
        print('Golem est maintenant connecté à IB')
    return

def disconX():
    ib.disconnect()
    if ib.isConnected() == False:
        print('Golem est maintenant déconnecté de IB')
    return

def relevpoz():
    global log
    CPT = []
    QT = []
    TYPE = []
    STOCK = []
    TICK = []
    AVGCOST = []
    CONID = []
    globalpoz = ib.positions(account=compte)
    
    if globalpoz == []:
        print("Il n'y a aucune position en cours actuellement.")
        
        log = log + "Il n'y a aucune position en cours actuellement. \n"
        
        poz = pd.DataFrame({'ALERTE': [101]}) 
    else:
        print('Il y a actuellement ',len(globalpoz),'positions en cours')
        
        log = log + 'Il y a actuellement ' + str(len(globalpoz)) + 'positions en cours \n'
        
        for i in range(0,len(globalpoz)):
            CPT.append(globalpoz[i][0])
            TYPE.append(str(globalpoz[i][1]).split("(")[0])
            if str(globalpoz[i][1]).split("(")[0] == 'Future':
                TICK.append(str(globalpoz[i][1]).split("'")[1])
            elif str(globalpoz[i][1]).split("(")[0] == 'Forex':
                TICK.append(str(globalpoz[i][1]).split("'")[1])
            elif str(globalpoz[i][1]).split("(")[0] == 'Stock':
                TICK.append(str(globalpoz[i][1]).split("'")[1])
            else:
                TICK.append('NA')

            QT.append(globalpoz[i][2])
            AVGCOST.append(globalpoz[i][3])
            CONID.append(ib.positions(account=compte)[i][1].conId)
            poz = pd.DataFrame()
            
            poz['Compte'] = CPT
            poz['Type'] = TYPE
            poz['conId'] = CONID
            poz['Ticker'] = TICK
            poz['Quantité'] = QT
            poz['Average Cost'] = AVGCOST
    return(poz)

def price_close(LaBase,ticker):
    
    price = round(LaBase[LaBase['Symbol']==ticker].iloc[-1]['Close'],2)
        
    return(price)


def CLOSE_SHORT(contract,quantity,price,ticker):
    
    ib.qualifyContracts(contract)
    order = LimitOrder('BUY', quantity, price,account=compte)
    trade = ib.placeOrder(contract, order)
    print(trade,'\n \n')        
    return()

def CLOSE_LONG(contract,quantity,price,ticker):
    
    ib.qualifyContracts(contract)
    order = LimitOrder('SELL', quantity, price,account=compte)
    trade = ib.placeOrder(contract, order)
    print(trade,'\n \n')     
    return()

def OPEN_LONG(contract,quantity,price,ticker):
 
    ib.qualifyContracts(contract)
    order = LimitOrder('BUY', quantity, price,account=compte)
    trade = ib.placeOrder(contract, order)
    print(trade,'\n \n')  
    return()

def OPEN_SHORT(contract,quantity,price,ticker):

    ib.qualifyContracts(contract)
    order = LimitOrder('SELL', quantity, price,account=compte)
    trade = ib.placeOrder(contract, order)
    print(trade,'\n \n') 
    return()

In [16]:
# try:
try:
    disconX()
except:
    time.sleep(6)
    conX()

os.system('say "Béchème Hachème nahassé vénatsliahh"')
time.sleep(3)
conX()
time.sleep(3)
if ib.isConnected() == False:
    conX()

today = str(dt.date.today())[:5]+str(dt.date.today())[5:7]+str(dt.date.today())[7:10]
yesterday = str(dt.date.today()-dt.timedelta(1))[:5]+str(dt.date.today()-dt.timedelta(1))[5:7]+str(dt.date.today()-dt.timedelta(1))[7:10]

poz = relevpoz()


foo = 0
ACTION = []
ASKED_PRICE = []
WHEN = []
OL = []
CL = []
OS = []
CS = []

try:
    signals['Tradé']
except:
    signals['Tradé'] = 'NA'
if (signals[signals['Tradé']=='NA'].shape[0] == signals.shape[0]):
    print('1er passage')
else:

    foo = signals[signals['Tradé']=='NA'].index[0]
    print('On reprend le passage en ligne ',foo)

try:
    pactol = float(str(str([v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']).split()[2]).split("'")[1])
except:
    print('Montant du pactol non récupéré')
    ib.isConnected()

print(Back.BLUE,Fore.YELLOW,'Les fonds disponibles sur le compte sont de : $',pactol,Style.RESET_ALL)
mise = pactol * 0.02
tmps55=time.time()
for i in range (foo,signals.shape[0]):

    horo = signals.iloc[i,0]
    ticker = signals.iloc[i,1]
    openlong = signals.iloc[i,2]
    closelong = signals.iloc[i,3]
    openshort = signals.iloc[i,4]
    closeshort = signals.iloc[i,5]
    contract = Stock(ticker,'SMART','USD')
    price = price_close(LaBase,ticker)

    en_poz = 0
    if poz.iloc[0,0] != 101:
        if len(poz[ticker == poz.iloc[:]['Ticker']]) > 1:
            exit("Il y a plus d'une position pour le ticker ",ticker)
        elif len(poz[ticker == poz.iloc[:]['Ticker']]) == 1:
            indeX = int(str(str(poz[ticker == poz.iloc[:]['Ticker']].index).split('[')[1]).split(']')[0])
            quantity = abs(poz.iloc[indeX]['Quantité'])

            if poz.iloc[indeX]['Quantité'] < 0:
                en_poz = -1
            if poz.iloc[indeX]['Quantité'] > 0:
                en_poz = 1

    if openlong == 1 and en_poz == 0:
        print('')
        price = round(price * 1.002,2)
        quantity = round(mise/price)
        print(Fore.GREEN,'Open Long',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', price,'\n')

        log = log + '    Open Long ' + str(quantity) + ' ' +  str(ticker) + ' à ' + str(price) + '\n'

        WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
        ASKED_PRICE.append(price)
        ACTION.append('Open Long')
        OL.append(1)
        OPEN_LONG(contract,quantity,price,ticker)


    elif openshort == 1 and en_poz == 0:
        print('')
        price = round(price * 0.998,2)
        quantity = round(mise/price)
        print(Fore.GREEN,'Open Short',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', price,'\n')

        log = log +  '    Open Short ' + str(quantity) + ' ' +  str(ticker) + ' à ' + str(price) + '\n'

        WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
        ASKED_PRICE.append(price)
        ACTION.append('Open Short')
        OS.append(1)
        OPEN_SHORT(contract,quantity,price,ticker)


    elif closelong == 1 and en_poz == 1:
        print('')
        price = round(price * 0.998,2)
        print(Fore.RED,'Close Long',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', str(price),'\n')

        log = log + '    Close Long ' + str(quantity) + ' ' +  str(ticker) + ' à ' + str(price) + '\n'

        WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
        ASKED_PRICE.append(price)
        ACTION.append('Close Long')
        CL.append(1)
        CLOSE_LONG(contract,quantity,price,ticker)


    elif closeshort == 1 and en_poz == -1:
        print('')
        price = round(price * 1.002,2)
        print(Fore.RED,'Close Short',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', price,'\n')

        log = log + 'Close Short ' + str(quantity) + ' ' +  str(ticker) + ' à ' + str(price) + '\n'

        WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
        ASKED_PRICE.append(price)
        ACTION.append('Close Short')
        CS.append(1)
        CLOSE_SHORT(contract,quantity,price,ticker)


    else:
        WHEN.append('NA')
        ASKED_PRICE.append('NA')
        ACTION.append('NA')

    signals.iloc[i,7]='Traité'
    #except:
     #   print('souci boucle ',i,' pour le ticker ',ticker)
      #  continue

traded = signals
traded['Action'] = ACTION
traded['Quand'] = WHEN
traded['Prix limite'] = ASKED_PRICE
nownow = time.strftime("%d_%m_%Y_%H_%M_%S", time.localtime())
traded[(traded['Open Long']==1) | (traded['Close Long']==1)].to_csv('ORDRES/OrdresPasses_'+nownow+'.csv')
poz.to_csv('RELEVE_POZ/RelevPoz_'+nownow+'.csv')
tmps22=round(time.time()-tmps55,2)
print("\n Ensemble des opérations du jour effectué en %f" %tmps22,'seconds \n')

log = log+'\n'+today+': \n La NAV est de : '+str(pactol)+'$ \n Il y a eu '+str(sum(OL))+' Open Long, '+str(sum(OS))+' Open Short, '+str(sum(CL))+' Close Long, et '+\
str(sum(CS))+' Close Short. \n \n \n'

print(log)
fichier = open("log.txt", "a")
fichier.write(log)
fichier.close()
disconX()
'''except:
    
    try:
        disconX()
    except:
        time.sleep(6)
        conX()

    os.system('say "Béchème Hachème nahassé vénatsliahh"')
    time.sleep(3)
    conX()
    time.sleep(3)
    if ib.isConnected() == False:
        conX()

    today = str(dt.date.today())[:5]+str(dt.date.today())[5:7]+str(dt.date.today())[7:10]
    yesterday = str(dt.date.today()-dt.timedelta(1))[:5]+str(dt.date.today()-dt.timedelta(1))[5:7]+str(dt.date.today()-dt.timedelta(1))[7:10]

    poz = relevpoz()


    foo = 0
    ACTION = []
    ASKED_PRICE = []
    WHEN = []
    OL = []
    CL = []
    OS = []
    CS = []

    try:
        signals['Tradé']
    except:
        signals['Tradé'] = 'NA'
    if (signals[signals['Tradé']=='NA'].shape[0] == signals.shape[0]):
        print('1er passage')
    else:

        foo = signals[signals['Tradé']=='NA'].index[0]
        print('On reprend le passage en ligne ',foo)

    try:
        pactol = float(str(str([v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']).split()[2]).split("'")[1])
    except:
        print('Montant du pactol non récupéré')
        ib.isConnected()

    print(Back.BLUE,Fore.YELLOW,'Les fonds disponibles sur le compte sont de : $',pactol,Style.RESET_ALL)
    mise = pactol * 0.02
    tmps55=time.time()
    for i in range (foo,signals.shape[0]):

        horo = signals.iloc[i,0]
        ticker = signals.iloc[i,1]
        openlong = signals.iloc[i,2]
        closelong = signals.iloc[i,3]
        openshort = signals.iloc[i,4]
        closeshort = signals.iloc[i,5]
        contract = Stock(ticker,'SMART','USD')
        price = price_close(LaBase,ticker)

        en_poz = 0
        if poz.iloc[0,0] != 101:
            if len(poz[ticker == poz.iloc[:]['Ticker']]) > 1:
                exit("Il y a plus d'une position pour le ticker ",ticker)
            elif len(poz[ticker == poz.iloc[:]['Ticker']]) == 1:
                indeX = int(str(str(poz[ticker == poz.iloc[:]['Ticker']].index).split('[')[1]).split(']')[0])
                quantity = abs(poz.iloc[indeX]['Quantité'])

                if poz.iloc[indeX]['Quantité'] < 0:
                    en_poz = -1
                if poz.iloc[indeX]['Quantité'] > 0:
                    en_poz = 1

        if openlong == 1 and en_poz == 0:
            print('')
            price = round(price * 1.002,2)
            quantity = round(mise/price)
            print(Fore.GREEN,'Open Long',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', price,'\n')

            log = log + 'Open Long sur le ticker ' + ticker + ' basé sur un close limité à ' + str(price) + '\n'

            WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
            ASKED_PRICE.append(price)
            ACTION.append('Open Long')
            OL.append(1)
            OPEN_LONG(contract,quantity,price,ticker)


        elif openshort == 1 and en_poz == 0:
            print('')
            price = round(price * 0.998,2)
            quantity = round(mise/price)
            print(Fore.GREEN,'Open Short',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', price,'\n')

            log = log + 'Open Short sur le ticker ' + ticker + ' basé sur un close limité à ' + str(price) + '\n'

            WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
            ASKED_PRICE.append(price)
            ACTION.append('Open Short')
            OS.append(1)
            OPEN_SHORT(contract,quantity,price,ticker)


        elif closelong == 1 and en_poz == 1:
            print('')
            price = round(price * 0.998,2)
            print(Fore.RED,'Close Long',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', str(price),'\n')

            log = log + 'Close Long sur le ticker ' + ticker + ' basé sur un close limité à ' + str(price) + '\n'

            WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
            ASKED_PRICE.append(price)
            ACTION.append('Close Long')
            CL.append(1)
            CLOSE_LONG(contract,quantity,price,ticker)


        elif closeshort == 1 and en_poz == -1:
            print('')
            price = round(price * 1.002,2)
            print(Fore.RED,'Close Short',Style.RESET_ALL,' sur le ticker ',ticker,' basé sur un close limité à ', price,'\n')

            log = log + 'Close Short sur le ticker ' + ticker + ' basé sur un close limité à ' + str(price) + '\n'

            WHEN.append(time.strftime("%d-%m-%Y %H:%M:%S", time.localtime()))
            ASKED_PRICE.append(price)
            ACTION.append('Close Short')
            CS.append(1)
            CLOSE_SHORT(contract,quantity,price,ticker)


        else:
            WHEN.append('NA')
            ASKED_PRICE.append('NA')
            ACTION.append('NA')

        signals.iloc[i,7]='Traité'
        #except:
         #   print('souci boucle ',i,' pour le ticker ',ticker)
          #  continue

    traded = signals
    traded['Action'] = ACTION
    traded['Quand'] = WHEN
    traded['Prix limite'] = ASKED_PRICE
    nownow = time.strftime("%d_%m_%Y_%H_%M_%S", time.localtime())
    traded[(traded['Open Long']==1) | (traded['Close Long']==1)].to_csv('ORDRES/OrdresPasses_'+nownow+'.csv')
    poz.to_csv('RELEVE_POZ/RelevPoz_'+nownow+'.csv')
    tmps22=round(time.time()-tmps55,2)
    print("\n Ensemble des opérations du jour effectué en %f" %tmps22,'seconds \n')

    log = '\n'+today+': \n La NAV est de : '+str(pactol)+'$ \n Il y a eu '+str(sum(OL))+' Open Long, '+str(sum(OS))+' Open Short, '+str(sum(CL))+' Close Long, et '+\
    str(sum(CS))+' Close Short.\n'+log+'\n \n \n'

    print(log)
    fichier = open("log.txt", "a")
    fichier.write(log)
    fichier.close()
    disconX()'''

Golem est maintenant déconnecté de IB
Golem est maintenant connecté à IB
Il y a actuellement  139 positions en cours
1er passage
  Les fonds disponibles sur le compte sont de : $ 494979.7359 

 Open Short   sur le ticker  BAC  basé sur un close limité à  34.61 

Trade(contract=Stock(conId=10098, symbol='BAC', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='BAC', tradingClass='BAC'), order=LimitOrder(orderId=19992, clientId=1, action='SELL', totalQuantity=286.0, lmtPrice=34.61, account='DU1507426', conditions=[], softDollarTier=SoftDollarTier()), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 12, 13, 12, 11, 47, 846106, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')]) 
 


 Close Long   sur le ticker  DISCA  basé sur un close limité à  32.3 

Trade(contract=Stock(conId=53855036, symbol='DISCA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='DISCA', tradingClas

'except:\n    \n    try:\n        disconX()\n    except:\n        time.sleep(6)\n        conX()\n\n    os.system(\'say "Béchème Hachème nahassé vénatsliahh"\')\n    time.sleep(3)\n    conX()\n    time.sleep(3)\n    if ib.isConnected() == False:\n        conX()\n\n    today = str(dt.date.today())[:5]+str(dt.date.today())[5:7]+str(dt.date.today())[7:10]\n    yesterday = str(dt.date.today()-dt.timedelta(1))[:5]+str(dt.date.today()-dt.timedelta(1))[5:7]+str(dt.date.today()-dt.timedelta(1))[7:10]\n\n    poz = relevpoz()\n\n\n    foo = 0\n    ACTION = []\n    ASKED_PRICE = []\n    WHEN = []\n    OL = []\n    CL = []\n    OS = []\n    CS = []\n\n    try:\n        signals[\'Tradé\']\n    except:\n        signals[\'Tradé\'] = \'NA\'\n    if (signals[signals[\'Tradé\']==\'NA\'].shape[0] == signals.shape[0]):\n        print(\'1er passage\')\n    else:\n\n        foo = signals[signals[\'Tradé\']==\'NA\'].index[0]\n        print(\'On reprend le passage en ligne \',foo)\n\n    try:\n        pactol =

In [17]:
import smtplib
import mimetypes
from email.message import EmailMessage

In [18]:
filename1 = ('SIGNALS/SIGNALS_'+stamp+'.csv')
filename2 = ('ORDRES/OrdresPasses_'+nownow+'.csv')
filename3 = ('RELEVE_POZ/RelevPoz_'+nownow+'.csv')
filename4 = ("log.txt")

os.system('say "Envoi du mail"')
msg = EmailMessage()
msg.set_content('Hi. Here are the reports of what was trading '+today+' on account '+compte+' for signals from '+stamp)


ctype, encoding = mimetypes.guess_type(filename1)
ctype, encoding = mimetypes.guess_type(filename2)
ctype, encoding = mimetypes.guess_type(filename3)
ctype, encoding = mimetypes.guess_type(filename4)

if ctype is None or encoding is not None:
    # No guess could be made, or the file is encoded (compressed), so
    # use a generic bag-of-bits type.
    ctype = 'application/octet-stream'
maintype, subtype = ctype.split('/', 1)
with open(filename1, 'rb') as fp:
    msg.add_attachment(fp.read(),
                       maintype=maintype,
                       subtype=subtype,
                       filename=filename1)
with open(filename2, 'rb') as fp:
    msg.add_attachment(fp.read(),
                       maintype=maintype,
                       subtype=subtype,
                       filename=filename2)

with open(filename3, 'rb') as fp:
    msg.add_attachment(fp.read(),
                       maintype=maintype,
                       subtype=subtype,
                       filename=filename3)
with open(filename4, 'rb') as fp:
    msg.add_attachment(fp.read(),
                       maintype=maintype,
                       subtype=subtype,
                       filename=filename4)

msg['Subject'] = ' Automatic report #'+compte+'. Signals from '+stamp+' traded on '+today
msg['From'] = 'jyss@dag26.com'
msg['To'] = 'stephane@dag26.com' #'GM@css-europe.co.uk'
msg['Cc'] = 'jyss@dag26.com' #,'billy@dag26.com','stephane@dag26.com'

# Send the message via our own SMTP server.
s = smtplib.SMTP('smtp.free.fr')
s.send_message(msg)
s.quit()    
print('Mail envoyé')
os.system('say "Mail envoyé"')
time.sleep(3)

os.system('say "Tout, tout, tout est fini entre nous... Allez... Au revoir batard, et à demain, dans lcamboui"')

Mail envoyé


0

In [ ]:
# signals = signals.drop(['Tradé'],axis=1)

In [ ]:
# PYPL, DE, ZION

In [ ]:
df1 = df
df2 = pd.read_csv('LaBasePrep.csv')
df2 = df2[df2['Symbol']=='AAPL']
df2 = df2.drop(['Symbol'],axis=1)
df2 = df2.iloc[-771:,:]
df2.index = pd.to_datetime(df2['Date'],format='%Y-%m-%d')
df2 = df2.drop(['Date'],axis=1)
df1.shape,df2.shape

In [ ]:
print('Longueur des df1 & df2 :', df1.shape[0],' & ',df2.shape[0],' = ',abs(df1.shape[0]-df2.shape[0]),len(pd.concat([df1, df2]).loc[df1.index.symmetric_difference(df2.index)]))
pd.concat([df1, df2]).loc[df1.index.symmetric_difference(df2.index)]

In [ ]:
df1.index.symmetric_difference(df2.index)

In [ ]:
df2[df2.index=='2019-12-06']